In [113]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [114]:
import requests
import pandas as pd
from datetime import datetime, date, timedelta
import numpy as np
import re
import math

In [115]:
data = pd.read_csv('/content/gdrive/My Drive/ABD_Lab_dataset/hh_parsed_4.csv', delimiter=',')
#data = data.drop(columns=['Unnamed: 0'])
del data['Unnamed: 0']
del data['published_at']
del data['responsibility']
del data['requirement']

df2 = data.loc[data['area'].str.lower().str.contains('москва|санкт-петербург')]
#data = data.drop(data[data['area'].str.lower().str.contains('москва|санкт-петербург')].index)
data

,name,area,minSalary,maxSalary,employer,schedule,category,days
0,Системный архитектор НИОКР (вычислительная тех...,Москва,120000.0,143308.0,Воентелеком,Полный день,Другое,42.0
1,Java Developer Trainee,Санкт-Петербург,86685.0,121062.0,Andersen,Полный день,Другое,42.0
2,Java Developer Trainee,Казань,57367.0,91784.0,Andersen,Полный день,Другое,42.0
3,Начальник отдела развития,Волгодонск,29600.0,59500.0,Пятёрочка,Полный день,Другое,42.0
4,Главный инженер контроля качества тестирования,Москва,98585.0,143308.0,Центральный банк Российской Федерации,Полный день,Другое,42.0
...,...,...,...,...,...,...,...,...
48040,Монтажник ЛКС,Туринск,20000.0,NaN,"К Телеком,ООО",Полный день,Другое,12.0
48041,Программист JS,Ярославль,75000.0,120000.0,Тензор,Гибкий график,Программист,12.0
48042,Программист Python,Ярославль,61000.0,90000.0,Тензор,Гибкий график,Программист,12.0
48043,Оператор ЭВМ (ПК),Ульяновск,30000.0,35000.0,Альфа Люкс,Полный день,Оператор,12.0




---


ЧАСТЬ 1
Выполнить сокращение датасета с помощью сокращения размерности: удалить вакансии, у которых больше 70% пропусков по признакам.


---



In [116]:

#data = data.loc[:, data.isnull().mean() < .7]
#data
print(len(data))
data.dropna(axis=0, thresh=0.3, inplace=True)
print(len(data))


48045
48045




---


Часть 2.

Решить задачу классификации вакансий по группам, которые были определены вами в предыдущих лабораторных. Что нужно сделать:


---



In [117]:
def classify(data):
  data = data.dropna(how='any');
  data.loc[data['name'].str.lower().str.contains('angular'), 'name'] = 'Angular'
  data.loc[data['name'].str.lower().str.contains('node'), 'name'] = 'Node'
  data.loc[data['name'].str.lower().str.contains('django'), 'name'] = 'Django'
  data.loc[data['name'].str.lower().str.contains('php'), 'name'] = 'PHP'
  data.loc[data['name'].str.lower().str.contains('vue'), 'name'] = 'Vue'

  data['minSalary'] = data.groupby(['name','area'], sort=False)['minSalary'].apply(lambda x: x.ffill().bfill())
  data['maxSalary'] = data.groupby(['name','area'], sort=False)['maxSalary'].apply(lambda x: x.ffill().bfill())
  data = data[data['name'].str.contains('Angular|Django|Node|PHP|Vue')]

  return data

In [118]:
data = classify(data)
data

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,name,area,minSalary,maxSalary,employer,schedule,category,days
152,PHP,Санкт-Петербург,100000.0,121062.0,Международные Образовательные Проекты,Полный день,Другое,42.0
418,Node,Минск,4113.0,8936.0,Godel Technologies Europe,Полный день,Другое,41.0
419,PHP,Москва,110929.0,140000.0,Staff-UP Consulting Group,Удаленная работа,Программист,41.0
439,Node,Санкт-Петербург,86685.0,121062.0,ZiMAD,Удаленная работа,Другое,41.0
442,Node,Калуга,52052.0,74973.0,ZiMAD,Удаленная работа,Другое,41.0
...,...,...,...,...,...,...,...,...
47825,Angular,Казань,57367.0,91784.0,Гет Гейн,Полный день,Другое,12.0
47830,Node,Новосибирск,70000.0,120000.0,Logistic Eco Systems,Гибкий график,Другое,12.0
47847,Node,Санкт-Петербург,86685.0,121062.0,the_covert,Гибкий график,Другое,12.0
47918,PHP,Иркутск,50000.0,80000.0,ИОТ,Полный день,Программист,12.0




---


2.2 с помощью LabelEncoder закодировать названия классов и вывести в файл в формате: № + название класса;

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html


---



In [119]:
le = preprocessing.LabelEncoder()
le.fit(data.name)
for i in le.transform(list(le.classes_)):
  print(i, le.inverse_transform([i]))

0 ['Angular']
1 ['Django']
2 ['Node']
3 ['PHP']
4 ['Vue']


In [120]:
le2 = preprocessing.LabelEncoder()
le2.fit(data.area)
le3 = preprocessing.LabelEncoder()
le3.fit(data.employer)
le4 = preprocessing.LabelEncoder()
le4.fit(data.schedule)
le5 = preprocessing.LabelEncoder()
le5.fit(data.category)


data['area'] = le2.transform(data['area'])
data['employer'] = le3.transform(data['employer'])
data['schedule'] = le4.transform(data['schedule'])
data['category'] = le5.transform(data['category'])
data

,name,area,minSalary,maxSalary,employer,schedule,category,days
152,PHP,81,100000.0,121062.0,643,1,1,42.0
418,Node,54,4113.0,8936.0,131,1,1,41.0
419,PHP,56,110929.0,140000.0,298,2,3,41.0
439,Node,81,86685.0,121062.0,346,2,1,41.0
442,Node,35,52052.0,74973.0,346,2,1,41.0
...,...,...,...,...,...,...,...,...
47825,Angular,33,57367.0,91784.0,483,1,1,12.0
47830,Node,63,70000.0,120000.0,182,0,1,12.0
47847,Node,81,86685.0,121062.0,359,0,1,12.0
47918,PHP,30,50000.0,80000.0,541,1,3,12.0


**разделить исходный датасет на обучающую и тестирующую выборки в соотношении 70% и 30%;**

In [121]:
train = data.iloc[:int(len(data)*0.7)]
test = data.iloc[int(len(data)*0.7)+1:]
train

,name,area,minSalary,maxSalary,employer,schedule,category,days
152,PHP,81,100000.0,121062.0,643,1,1,42.0
418,Node,54,4113.0,8936.0,131,1,1,41.0
419,PHP,56,110929.0,140000.0,298,2,3,41.0
439,Node,81,86685.0,121062.0,346,2,1,41.0
442,Node,35,52052.0,74973.0,346,2,1,41.0
...,...,...,...,...,...,...,...,...
33835,PHP,81,59770.0,77429.0,103,1,2,19.0
33839,Angular,84,51405.0,92760.0,903,1,1,19.0
33879,Node,26,44300.0,65143.0,549,1,1,19.0
33913,Vue,63,72414.0,98281.0,792,1,1,19.0


In [122]:
train_labels = le.transform(train.name)
test_labels = le.transform(test.name)

del train['name']
del test['name']
train_labels

array([3, 2, 3, ..., 2, 4, 3])

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
clf = LogisticRegression()
scores = cross_val_score(clf, train, train_labels)
print("____________________________________________")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("____________________________________________")

scores = cross_val_score(clf, test, test_labels)
print("____________________________________________")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("____________________________________________")




/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


____________________________________________
Accuracy: 0.64 (+/- 0.01)
____________________________________________
____________________________________________
Accuracy: 0.69 (+/- 0.02)
____________________________________________


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [124]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
scores = cross_val_score(neigh, train, train_labesl)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = cross_val_score(neigh, test, test_labesl)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.61 (+/- 0.12)
Accuracy: 0.59 (+/- 0.05)


In [125]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0)
scores = cross_val_score(clf, train, train_labesl)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = cross_val_score(clf, test, test_labesl)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.42 (+/- 0.36)
Accuracy: 0.52 (+/- 0.37)


**Последняя модель чет совсем мало выдала, а в первой какие-то варнинги (что вроде не очень хорошо, но как это исправить я так и не понял), поэтому возьмем вторую модель на K-соседей**



---

**Начало 2.2**

---



In [126]:
df2 = classify(df2)
df2

,name,area,minSalary,maxSalary,employer,schedule,category,days
152,PHP,Санкт-Петербург,100000.0,121062.0,Международные Образовательные Проекты,Полный день,Другое,42.0
419,PHP,Москва,110929.0,140000.0,Staff-UP Consulting Group,Удаленная работа,Программист,41.0
439,Node,Санкт-Петербург,86685.0,121062.0,ZiMAD,Удаленная работа,Другое,41.0
458,Node,Санкт-Петербург,86685.0,121062.0,Murano Software,Гибкий график,Другое,41.0
740,PHP,Москва,98585.0,143308.0,МИА Россия сегодня,Полный день,Другое,41.0
...,...,...,...,...,...,...,...,...
47677,PHP,Москва,130000.0,170000.0,Бэш-Тудэй,Полный день,Другое,12.0
47691,PHP,Санкт-Петербург,86685.0,121062.0,ЭдСервер,Полный день,Другое,12.0
47770,PHP,Санкт-Петербург,86685.0,121062.0,ЭдСервер,Полный день,Другое,12.0
47783,PHP,Москва,180000.0,260000.0,Americor Funding Inc,Удаленная работа,Другое,12.0


**Немного не уверен на счет того надо было ли дропать `category` но на всякий соучай этот признак оставил**

In [130]:
le = preprocessing.LabelEncoder()
le.fit(df2.name)
for i in le.transform(list(le.classes_)):
  print(i, le.inverse_transform([i]))

le2 = preprocessing.LabelEncoder()
le2.fit(df2.area)
le3 = preprocessing.LabelEncoder()
le3.fit(df2.employer)
le4 = preprocessing.LabelEncoder()
le4.fit(df2.schedule)
le5 = preprocessing.LabelEncoder()
le5.fit(df2.category)



df2['area'] = le2.transform(df2['area'])
df2['employer'] = le3.transform(df2['employer'])
df2['schedule'] = le4.transform(df2['schedule'])
df2['category'] = le5.transform(df2['category'])
df2

0 ['Angular']
1 ['Django']
2 ['Node']
3 ['PHP']
4 ['Vue']


,name,area,minSalary,maxSalary,employer,schedule,category,days
152,PHP,1,100000.0,121062.0,322,1,1,42.0
419,PHP,0,110929.0,140000.0,153,2,3,41.0
439,Node,1,86685.0,121062.0,179,2,1,41.0
458,Node,1,86685.0,121062.0,104,0,1,41.0
740,PHP,0,98585.0,143308.0,314,1,1,41.0
...,...,...,...,...,...,...,...,...
47677,PHP,0,130000.0,170000.0,227,1,1,12.0
47691,PHP,1,86685.0,121062.0,438,1,1,12.0
47770,PHP,1,86685.0,121062.0,438,1,1,12.0
47783,PHP,0,180000.0,260000.0,14,2,1,12.0


In [131]:
data_labels = le.transform(data.name)
del data['name']
df2_labels = le.transform(df2.name)
del df2['name']

In [132]:

neigh = KNeighborsClassifier()
scores = cross_val_score(neigh, data, data_labels)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.60 (+/- 0.05)


In [133]:
neigh.fit(data, data_labels)
predictions = neigh.predict(df2)
predictions

array([3, 3, 2, 0, 3, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2,
       3, 3, 2, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 4, 3, 2, 1, 2,
       2, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 0, 3, 3, 4, 4,
       3, 0, 2, 4, 3, 3, 3, 3, 2, 3, 0, 3, 3, 3, 3, 0, 3, 2, 2, 3, 2, 2,
       3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3,
       2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3,
       3, 3, 3, 2, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 2, 3,
       3, 2, 3, 3, 3, 0, 3, 2, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 1,
       3, 3, 3, 3, 2, 2, 3, 3, 0, 0, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 0,
       3, 3, 3, 3, 3, 3, 3, 3, 2, 0, 3, 3, 3, 3, 3,

**вывести в csv-файл результаты предсказания в формате по столбцам: “Предсказанная моделью группа”, “Определенная по алгоритму группа”, [Признаки вакансий].**

In [134]:
da = pd.DataFrame()
da["Предсказанная моделью групп"] = le.inverse_transform(predictions)
da["Определенная по алгоритму группа"] = le.inverse_transform(df2_labels)


In [135]:
df2['area'] = le2.inverse_transform(df2['area'])
df2['employer'] = le3.inverse_transform(df2['employer'])
df2['schedule'] = le4.inverse_transform(df2['schedule'])
df2['category'] = le5.inverse_transform(df2['category'])
da["Признаки вакансий"] = df2.values.tolist()

In [136]:
da.to_csv("da.csv")

In [137]:
pd.set_option('display.max_rows', None)
da

,Предсказанная моделью групп,Определенная по алгоритму группа,Признаки вакансий
0,PHP,PHP,"[1, 100000.0, 121062.0, 322, 1, Другое, 42.0]"
1,PHP,PHP,"[0, 110929.0, 140000.0, 153, 2, Программист, 4..."
2,Node,Node,"[1, 86685.0, 121062.0, 179, 2, Другое, 41.0]"
3,Angular,Node,"[1, 86685.0, 121062.0, 104, 0, Другое, 41.0]"
4,PHP,PHP,"[0, 98585.0, 143308.0, 314, 1, Другое, 41.0]"
5,Node,PHP,"[0, 110929.0, 300000.0, 406, 1, Программист, 4..."
6,PHP,PHP,"[0, 110929.0, 110000.0, 212, 2, Программист, 4..."
7,PHP,PHP,"[0, 98585.0, 143308.0, 431, 1, Другое, 41.0]"
8,PHP,Vue,"[0, 98585.0, 143308.0, 391, 1, Другое, 41.0]"
9,PHP,PHP,"[0, 110929.0, 145932.0, 211, 2, Программист, 4..."
